#Start: Data Preparation

simply start by reading the original data,, encode the Industry as label,, only use basic columns, remove TVC and ROE because they are highly correlated to TRC and ROA, respectively. drop rows with null columns, lazy to (properly) impute

In [9]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score

X_train, y_train = pd.read_csv("X_train.csv").drop(columns=['Unnamed: 0']), pd.read_csv("y_train.csv").drop(columns=['Unnamed: 0'])
X_test, y_test = pd.read_csv("X_test.csv").drop(columns=['Unnamed: 0']), pd.read_csv("y_test.csv").drop(columns=['Unnamed: 0'])
print(X_train.dtypes)

Industry        int64
MR            float64
MR.PR         float64
MR.Decile       int64
TVC           float64
TVC.PR        float64
TVC.Decile      int64
TRC           float64
TRC.PR        float64
TRC.Decile      int64
BAB           float64
BAB.PR        float64
BAB.Decile      int64
EV            float64
EV.PR         float64
EV.Decile       int64
P/B           float64
P/B.PR        float64
P/B.Decile      int64
PSR           float64
PSR.PR        float64
PSR.Decile      int64
ROE           float64
ROE.PR        float64
ROE.Decile      int64
ROA           float64
ROA.PR        float64
ROA.Decile      int64
C/A           float64
C/A.PR        float64
C/A.Decile      int64
D/A           float64
D/A.PR        float64
D/A.Decile      int64
PG            float64
PG.PR         float64
PG.Decile       int64
AG            float64
AG.PR         float64
AG.Decile       int64
dtype: object


In [10]:
from catboost import CatBoostRegressor
from pipeline_setup import prepare_model_pipeline
#contoh options1
model_class = CatBoostRegressor
model_params_dict = {"iterations":100, "depth":3, "learning_rate":0.01, "verbose":0}
model_pipeline = prepare_model_pipeline(column_mode="original",
                                        use_transformation=False,
                                        use_rfe=False,
                                        n_neighbors=None,
                                        n_clusters=None,
                                        n_features_to_select=None,
                                        model_class=model_class,
                                        model_params_dict=model_params_dict)

In [ ]:
cv_scores = cross_val_score(model_pipeline, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

In [15]:
print(cv_scores)

[0.08239437 0.07456885 0.07428744 0.07506599 0.07161115]


#Hyperparameter tuning

In [3]:
def objective(trial):
    X = data.drop(columns=["Yt.1M"])
    y = data["Yt.1M"]

    param = {
        "objective": "regression",
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "depth": trial.suggest_int("depth", 3, 8),
        "iterations": trial.suggest_int("iterations", 50, 150),
        "n_clusters": trial.suggest_int("n_clusters", 4, 8),
        "n_neighbors": trial.suggest_int("n_neighbors", 1, 5),
        "n_features_to_select": trial.suggest_int("n_features_to_select", 5, 15),
    }
    model = prepare_model_pipeline(iterations=param["iterations"],
                                depth=param["depth"],
                                lr=param["learning_rate"],
                                n_clusters=param["n_clusters"],
                                n_neighbors=param["n_neighbors"],
                                n_features_to_select=param["n_features_to_select"])
    cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)
    return -np.asanyarray(cv_scores).mean()

In [4]:
import optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=500)  # you can increase n_trials

c:\Users\user\miniconda3\envs\vrp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-04-15 00:12:03,012] A new study created in memory with name: no-name-0ffe985b-0e0b-4c40-97c1-237a1ddd5e29
[I 2025-04-15 00:23:56,692] Trial 0 finished with value: 0.12154035052916727 and parameters: {'learning_rate': 0.03986616253280931, 'depth': 7, 'iterations': 133, 'n_clusters': 8, 'n_neighbors': 3, 'n_features_to_select': 11}. Best is trial 0 with value: 0.12154035052916727.
[I 2025-04-15 00:24:51,998] Trial 1 finished with value: 0.12160487270037998 and parameters: {'learning_rate': 0.06087346864528601, 'depth': 3, 'iterations': 75, 'n_clusters': 7, 'n_neighbors': 3, 'n_features_to_select': 5}. Best is trial 0 with value: 0.12154035052916727.
[I 2025-04-15 00:25:56,931] Trial 2 finished with value: 0.11980021009610672 an

KeyboardInterrupt: 